# General Images Ripper
Salut! This notebook has the power to _rip images from almost anywhere in the web!_ Its kinda like `ripme.jar` but much more customizable and doesn't require to be updated once in a while (this notebook's up to me update it, unfortunately lol). This is a **_work in progress_**, and if you want this notebook to rip a specific site that you want, then you gotta figure that out yourself, unless you want to pay me then go to [contact me](https://arifhamed.com/contact). The functions here should suffice.

Requirements: 
- **Internet** connection
- **Google Account** 
- A **browser** to run this
- (Optional) **`curl`** (If you are running on Linux then it should be no problem). One of the download methods makes use of https://file.io, a command-line file transfer site. Very similar in function like https://wetransfer.com, but instead of email its for command-line.

Usage: 
- Run the first cell (mandatory)
- Run whatever cell that is labelled by its respective markdown block above it. 
- Sit back & relax. You can do other things while this notebook will do its magic. Google Colab will send a notification after its done (if you allowed it to do so through the browser)
- Once its done, the following would occur if your `dl_mode` in the `save_content()` function is set as:
  - `curl`
    - The `curl` command will be shown as a string in the output of the cell that has `dl_mode` set as `curl` in the `save_content()` function. Just **copy & paste** the string to your **terminal/console** of choice (works on **Ubuntu 22.04**, **fastest option**). The output files are uploaded to https://file.io for temporary storage, which will last for 24 hours, and it will delete itself after 1 download. There's much more options for uploading, check it out on their website.
  - `files` 
    - The output files will be normally downloaded through the browser (slow, but reliable if https://file.io becomes unstable or if Google Drive gets hacked)
  - `drive` 
    - This notebook may require you to mount your [**Google Drive**](https://drive.google.com) to this instance so that it can move the output files to your Google Drive instead. This is good if you are not on a _machine that has much space_ and if you have a _lot of space to spare in your drive_. Recommended if you either use **ChromeOS**, if you **don't immediately have your main machine with you**, or if you're very **cloud-centric**. 

In [ ]:
!pip install img2pdf

import time, pytz, os, sys, requests, codecs, json
from tqdm.notebook import tqdm
from google.colab import files
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import timedelta, datetime
from IPython.display import clear_output 
time_alpha = time.time()

# Essential to make sure webpages does not detect you as a bot (even though this
# is basically a bot). Does not work for those that require Javascript (WIP for 
# a workaround)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
def request_with_headers(url, sep="\n"): # Returns list
  return requests.get(url, headers=headers).content.decode().split(sep)

file_io_command = "" # For curl

def save_content(savename="output", type_mode="zip", dl_mode="files", pdf_args=""): 
  global file_io_command
  # clear_output() # Clears previous file_io_command if there was
  os.system(f"mkdir -p /content/{type_mode}")
  savename = f"/content/{type_mode}/{savename}.{type_mode}"
  if type_mode == "zip":
    os.system(f"zip -r {savename} /content/*.*")
  elif type_mode == "pdf":
    os.system(f"img2pdf /content/*.* -o {savename} --producer=\"Google Colab\"  {pdf_args}")
  
  # I really wanted to use `match` (released in Python 3.10) here 
  # but Google Colab only has Python 3.8.16 as of this typing
  if dl_mode == "files":
    files.download(savename)
  if dl_mode == "curl":
    obj_temp = json.loads(os.popen(f'curl -F "file=@{savename}" https://file.io').read())
    file_io_command += f"curl {obj_temp['link']} --output {obj_temp['name']}; "
  if dl_mode == "drive":
    os.system(f"mv {savename} /content/drive/MyDrive/{savename.split('/')[-1]}")

  # removes all if it is a file in /content
  for i in [x for x in os.listdir("/content") if os.path.isfile(x)]:
    try: os.remove(i)
    except: pass

!rm -rf /content/*

### Only run this cell if you want to use your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p ~/.ssh

## junji-ito.com
As of 15 Jan 2022, these 4 are publicly available on Junji Ito's site

In [ ]:
# the lists are set to variables so that tqdm knows how to much there is inside 
# in each iteration. 
for comic in tqdm(("tomie", "uzumaki", "gyo", "remina")):
  list_focus_i = ["http"+x.split("\"")[0] for x in request_with_headers(f"https://junji-ito.com/{comic}/",sep="http") if f"{comic}-chapter" in x][::-1]
  # n is album index
  for n, i in tqdm(enumerate(list_focus_i), total=len(list_focus_i), leave=False): 
    list_focus_j = list(dict.fromkeys([x.split("\"")[0] for x in request_with_headers(i, sep='property="og:image" content="')][1:]))
    for j in tqdm(list_focus_j, total=len(list_focus_j), leave=False):
      os.system(f"wget {j} -O {str(n).zfill(2)}_{j.split('/')[-1]}")
  save_content(f"junji-ito_{comic}", type_mode="pdf", dl_mode="curl", pdf_args=f" --title=\"{comic.capitalize()}\" --author=\"Junji Ito\" --subject=\"Horror\"")

# note for ari: reconsider this line below, cuz other people might want to use other forms of dl-ing
file_io_command

## readuzumaki.com
this one has the lost chapter, and filesize is smaller

In [ ]:
for i in tqdm(range(20)):
  for j in [x.split("\"")[0] for x in request_with_headers(f"https://readuzumaki.com/manga/uzumaki-chapter-{i+1}/", sep="<meta property=\"twitter:image\" content=\"") if x[:4] == "http"]:
    os.system(f"wget -O {str(i).zfill(2)}_{j.split('/')[-1]} {j}")

save_content("readuzumaki", type_mode="pdf", dl_mode="curl", pdf_args=f" --title=\"Uzumaki\" --author=\"Junji Ito\"")
file_io_command

## Carousell search (first page)


In [ ]:
# replace space with "+"
query = "thinkpad%2025"

# result = [y for y in [codecs.decode(x.split("\"")[0], "unicode-escape") for x in request_with_headers(f"https://www.carousell.sg/search/{query}", '":"') if x[:4] == "http"] if y[-4:] in (".png",".jpg","jpeg","webp",".gif",".mp4") and not(y.split("/")[-1][0].isnumeric())]
# result = ["http"+codecs.decode(x.split("\"")[0], "unicode-escape") for x in requests.get(f"https://www.carousell.sg/search/{query}", headers=headers).content.decode().split('http') if x[:20] == "s://www.carousell.sg"]
result = [codecs.decode(x.split("\"")[0], "unicode-escape") for x in request_with_headers(f"https://www.carousell.sg/search/{query}?addRecent=true&canChangeKeyword=true&includeSuggestions=true", 'http') if "anniversary" in x]


for i in result: 
  print(repr(i))
  # if not(os.path.isfile(i.split("/")[-1])):
  #   print(i)
  #   os.system(f"wget {i}")

# save_content(f"carousell_{query}.zip")

In [ ]:
# if drive is mounted, it is 1
if_drive_mounted = int(os.popen('[ -d "/content/drive/" ] && echo 1 || echo 0').read()[:-1])
if if_drive_mounted:
  print('bruh')
else:
  print('cunt')
if_drive_mounted